## Import Libraries

In [40]:
# Import libraries of interest.
# Numerical libraries
import sklearn #this is the main machine learning library
from sklearn.decomposition import PCA
import numpy as np #this is the numeric library
import scipy.stats as stats

#OS libraries
import urllib #this allows us to access remote files
import urllib2
import os
from collections import OrderedDict, defaultdict
import imp
import sys

#BCML libraries
from bcml.Parser import read_training as rt
from bcml.Parser import build_training as bt
from bcml.PubChemUtils import pubchempy_utils as pcp
from bcml.Chemoinformatics import chemofeatures as cf
from bcml.Train import train_model as tm
from bcml.Parser import read_testing as rtest
from bcml.Parser import build_testing as btest


# Visualization libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image

# Explanability libraries
import lime
import lime.lime_tabular

# Chemistry libraries
indigo = imp.load_source('indigo', 'indigo-python-1.2.3.r0-mac/indigo.py')
indigo_renderer = imp.load_source('indigo_renderer', 'indigo-python-1.2.3.r0-mac/indigo_renderer.py')
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from IPython.display import display, Image

## Train the model

In [41]:
class Train:
    def __init__(self):
        pass
    def load(self, filename, identifier):
        train = rt.Read(filename, identifier, id_name="PubChem")
        # Load SDFs from NCBI
        training_data = pcp.Collect(train.compounds, sdf=True, chunks=20, id_name='PubChem', predictors=train.predictors, proxy=None)
        training_data = cf.Update(training_data, remove_static=False)
        # Run PaDEL-Descriptor to extract 6150 substructural features
        training_data.update(padel=True)
        ids = [id for id, compound in dict.iteritems(OrderedDict(sorted(training_data.compound.items(), key=lambda t: t[0])))]
        # Create machine learning model using a Random Forest Classifier
        predictors = []
        names = []
        compounds = []
        training_compounds = OrderedDict(sorted(training_data.compound.items(), key=lambda t: t[0]))
        # Preprocess data
        for identifier, compound in dict.iteritems(training_compounds):
            predictors.append(training_compounds[identifier]['predictor'])
            compounds.append(training_compounds[identifier])
            names.append(identifier)
        predictor_values = np.array(predictors, '|S4').astype(np.float)
        #Generate predictor values: y
        predict = np.zeros(len(predictor_values), dtype=int)
        for i, value in np.ndenumerate(predictor_values):
            if value >= np.median(predictor_values):
                predict[i] = 1
        rows = len(predict)
        # Load the names of the features
        feature_names = []
        for compound in compounds:
            feature_names = sorted(compound['padelhash'].keys())
        for c in feature_names:
            if c == 'Name':
                feature_names.remove(c)
        columns = len(feature_names)
        data_table = np.zeros((rows, columns), dtype=np.float64)
        # Load the training values: X
        for index, value in np.ndenumerate(data_table):
            compound = compounds[index[0]]['padelhash']
            feature = list(feature_names)[index[1]]
            data_table[index] = float(compound[feature])
        self.data_table = data_table
        self.feature_names = feature_names
        self.compounds = compounds
        self.predict = predict
        self.predictor_values = predictor_values
        self.training_data = training_data
        self.training_compounds = training_compounds
        self.names = names
    def reduce_features(self):
        feature_list = np.genfromtxt("feature_list.txt", dtype="str", delimiter="\t", comments="%")
        feature_ids = [a for a, b in feature_list]
        feature_patterns = [b for a, b in feature_list]
        data_table = self.data_table
        names = self.names
        # Remove invariable features
        reduced_X = data_table[:,np.where(data_table.var(axis=0)!=0)[0]]
        reduced_feature_ids = [feature_ids[i] for i in np.where(data_table.var(axis=0)!=0)[0]]
        reduced_feature_patterns = [feature_patterns[i] for i in np.where(data_table.var(axis=0)!=0)[0]]
        rows = len(names)
        columns = len(reduced_feature_ids)
        reduced_data_table = np.zeros((rows, columns), dtype=np.float64)
        # Load the training values: X
        for index, value in np.ndenumerate(reduced_data_table):
            compound = self.compounds[index[0]]['padelhash']
            feature = list(reduced_feature_ids)[index[1]]
            reduced_data_table[index] = float(compound[feature])
        self.reduced_data_table = reduced_data_table
        self.reduced_feature_ids = reduced_feature_ids
        self.reduced_feature_patterns = reduced_feature_patterns
    def learn(self):
        self.clf = sklearn.ensemble.RandomForestClassifier(n_estimators=512, oob_score=True, n_jobs=-1, class_weight="balanced")
        self.clf.fit(X=self.reduced_data_table, y=self.predict)

## Evaluate classifier

In [42]:
class CrossValidate:
    def __init__(self, model):
        self.model = model
        self.clf = sklearn.ensemble.RandomForestClassifier(n_estimators=512, oob_score=True, n_jobs=-1, class_weight="balanced")
    def cross_validation(self):
        self.clf.fit(X=self.model.reduced_data_table, y=self.model.predict)
        def _run_cv(cv, clf, y, X):
            ys = []
            for train_idx, valid_idx in cv:
                clf.fit(X=X[train_idx], y=y[train_idx])
                cur_pred = clf.predict(X[valid_idx])
                ys.append((y[valid_idx], cur_pred))
            acc = np.fromiter(map(lambda tp: sklearn.metrics.accuracy_score(tp[0], tp[1]), ys), np.float)
            prec = np.fromiter(map(lambda tp: sklearn.metrics.precision_score(tp[0], tp[1]), ys), np.float)
            recall = np.fromiter(map(lambda tp: sklearn.metrics.recall_score(tp[0], tp[1]), ys), np.float)
            roc = np.fromiter(map(lambda tp: sklearn.metrics.roc_auc_score(tp[0], tp[1]), ys), np.float)
            print_line = ("\tAccuracy: %0.4f +/- %0.4f" % (np.mean(acc), np.std(acc) * 2))
            print(print_line)
            print_line = ("\tPrecision: %0.4f +/- %0.4f" % (np.mean(prec), np.std(prec) * 2))
            print(print_line)
            print_line = ("\tRecall: %0.4f +/- %0.4f" % (np.mean(recall), np.std(recall) * 2))
            print(print_line)
            print_line = ("\tReceiver Operator, AUC: %0.4f +/- %0.4f" % (np.mean(roc), np.std(roc) * 2))
            print(print_line)

# 50% hold-out very conservative uses half the data for training and half the data for testing
# Likely closer accuracy match to novel dataset

        cv = sklearn.cross_validation.StratifiedShuffleSplit(self.model.predict, n_iter=100, test_size=0.5)
        print("For 100 resamples at 50%")
        _run_cv(cv, self.clf, self.model.predict, self.model.reduced_data_table)

# 10-fold cross-validation, less conservative uses 90% of the data for training and 10% of the data for testing
# Likely closer accuracy between model and training data

        cv = sklearn.cross_validation.StratifiedKFold(self.model.predict, n_folds=10)
        print("For 10-fold cross validation")
        _run_cv(cv, self.clf, self.model.predict, self.model.reduced_data_table)
    def visualize(self, filename):
        plt.clf()
        sns.set_style("darkgrid")
        # Initialize the figure
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([-0.01, 1.01])
        plt.ylim([-0.01, 1.01])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Mean Receiver operating characteristic')
        tprs = []
        base_fpr = np.linspace(0, 1, 101)
        # Run 10 instances of 10X cross_validation
        for i in range(10):
            X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(self.model.reduced_data_table, self.model.predict, test_size=0.1)
            self.clf.fit(X_train, y_train)
            y_pred = self.clf.predict_proba(X_test)[:, 1]
            fpr, tpr, _ = sklearn.metrics.roc_curve(y_test, y_pred)
            plt.plot(fpr, tpr, 'b', alpha=0.15)
            tpr = np.interp(base_fpr, fpr, tpr)
            tpr[0] = 0.0
            tprs.append(tpr)
        # Get average and std for cross_validation
        tprs = np.array(tprs)
        mean_tprs = tprs.mean(axis=0)
        std = tprs.std(axis=0)
        tprs_upper = np.minimum(mean_tprs + std, 1)
        tprs_lower = mean_tprs - std
        #Plot multiple ROCs
        plt.plot(base_fpr, mean_tprs, 'b')
        plt.fill_between(base_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.3)
        plt.axes().set_aspect('equal')
        plt.savefig(filename)
        Image(filename = filename)    

## Load testing data

In [43]:
class Testing:
    def __init__(self):
        pass
    def load(self, filename):
        testing_data = pcp.Collect(local=filename, sdf=True)
        testing_data = cf.Update(testing_data, remove_static=False)
        testing_data.update(padel=True)
        testing_compounds = OrderedDict(sorted(testing_data.compound.items(), key=lambda t: t[0]))
        compounds = []
        for identifier, compound in dict.iteritems(testing_compounds):
            compounds.append(testing_compounds[identifier])
        #self.filename = filename
        #testing = rtest.Read(filename, id_name="PubChem")
        #testing_data = pcp.Collect(testing.compounds, sdf=True, chunks=20, id_name='PubChem', proxy=None)
        #pubchem_id_dict = {}
        #for compound in testing.compounds:
        #    pubchem_id_dict[compound['PubChem']] = compound['Name']
        #    testing_data = cf.Update(testing_data, remove_static=False)
        # Run PaDEL-Descriptor to extract 6150 substructural features
        #testing_data.update(padel=True)
        feature_names = []
        #testing_compounds = OrderedDict(sorted(testing_data.compound.items(), key=lambda t: t[0]))
        #compounds = []
        #for identifier, compound in dict.iteritems(testing_compounds):
        #    compounds.append(testing_compounds[identifier])
        # Load the names of the features
        feature_names = []
        for compound in compounds:
            feature_names = sorted(compound['padelhash'].keys())
        for c in feature_names:
            if c == 'Name':
                feature_names.remove(c)
        columns = len(feature_names)
        rows = len(testing_data.compound)
        test = np.zeros((rows, columns,), dtype=np.float64)
        compounds = []
        testing_names = []
        testing_data.compound = OrderedDict(sorted(testing_data.compound.items(), key=lambda t: t[0]))
        for id, compound in testing_data.compound.iteritems():
            compounds.append(compound)
            testing_names.append(id)
        self.testing_data = testing_data
        self.compounds = compounds
        self.testing_names = testing_names
        #self.pubchem_id_dict = pubchem_id_dict
        rows = len(testing_names)
        # Load the names of the features
        feature_names = []
        for compound in compounds:
            feature_names = sorted(compound['padelhash'].keys())
        for c in feature_names:
            if c == 'Name':
                feature_names.remove(c)
        columns = len(feature_names)
        testing_data_table = np.zeros((rows, columns), dtype=np.float64)
        # Load the training values: X
        for index, value in np.ndenumerate(testing_data_table):
            compound = compounds[index[0]]['padelhash']
            feature = list(feature_names)[index[1]]
            testing_data_table[index] = float(compound[feature])
        self.feature_names = feature_names
        self.testing_data_table = testing_data_table
    def reduce_features(self, train):
        feature_list = np.genfromtxt("feature_list.txt", dtype="str", delimiter="\t", comments="%")
        feature_ids = [a for a, b in feature_list]
        feature_patterns = [b for a, b in feature_list]
        data_table = self.testing_data_table
        names = self.testing_names
        # Remove invariable features
        reduced_feature_ids = train.reduced_feature_ids
        reduced_feature_patterns = train.reduced_feature_patterns
        rows = len(names)
        columns = len(reduced_feature_ids)
        reduced_data_table = np.zeros((rows, columns), dtype=np.float64)
        # Load the training values: X
        for index, value in np.ndenumerate(reduced_data_table):
            compound = self.compounds[index[0]]['padelhash']
            feature = list(reduced_feature_ids)[index[1]]
            reduced_data_table[index] = float(compound[feature])
        self.reduced_data_table = reduced_data_table
        self.reduced_feature_ids = reduced_feature_ids
        self.reduced_feature_patterns = reduced_feature_patterns 
    def learn(self, train):
        train.clf.fit(X=train.data_table, y=train.predict)
        print(train.clf.predict_proba(self.testing_data_table))
        print(self.testing_names)

## Evaluate the similarity of training and testing datasets

In [44]:
class VisualizeTesting():
    def __init__(self, train, testing):
        self.train = train
        self.testing = testing
    def pca(self):
        self.pca = PCA()
        self.pca.fit(self.train.data_table)
        self.pc_train = self.pca.transform(self.train.data_table)
        self.pc_testing = self.pca.transform(self.testing.testing_data_table)
    def viz_explained(self, filename):
        plt.clf()
        summed_variance = np.cumsum(self.pca.explained_variance_ratio_)
        plt.axhline(summed_variance[4], color="mediumpurple")
        barlist = plt.bar(range(25), summed_variance[:25], color="steelblue")
        thresh = np.where(summed_variance[:25] <= summed_variance[4])[0]
        for t in thresh:
            barlist[t].set_color('mediumpurple')
        plt.axhline(0.9, color="darkred")
        thresh = np.where(summed_variance[:25] >= 0.9)[0]
        for t in thresh:
            barlist[t].set_color('darkred')
        plt.title("Variance Explained by Each PC")
        plt.savefig(filename)
        Image(filename = filename)
    def viz_xx(self, filename):
        plt.clf()
        pc = self.pc_train
        pc_test = self.pc_testing
        f, axarr = plt.subplots(4, 4, sharex='col', sharey='row')
        axarr[0, 0].set_title("PC1")
        axarr[0, 1].set_title("PC2")
        axarr[0, 2].set_title("PC3")
        axarr[0, 3].set_title("PC4")
        axarr[0, 0].set_ylabel("PC2")
        axarr[1, 0].set_ylabel("PC3")
        axarr[2, 0].set_ylabel("PC4")
        axarr[3, 0].set_ylabel("PC5")
        axarr[0, 0].scatter(pc[:, 1], pc[:, 0])
        axarr[0, 0].scatter(pc_test[:, 1], pc_test[:, 0], color="red")
        axarr[1, 0].scatter(pc[:, 2], pc[:, 0])
        axarr[1, 0].scatter(pc_test[:, 2], pc_test[:, 0], color="red")
        axarr[2, 0].scatter(pc[:, 3], pc[:, 0])
        axarr[2, 0].scatter(pc_test[:, 3], pc_test[:, 0], color="red")
        axarr[3, 0].scatter(pc[:, 4], pc[:, 0])
        axarr[3, 0].scatter(pc_test[:, 4], pc_test[:, 0], color="red")
        axarr[0, 1].axis('off')
        axarr[1, 1].scatter(pc[:, 2], pc[:, 1])
        axarr[1, 1].scatter(pc_test[:, 2], pc_test[:, 1], color="red")
        axarr[2, 1].scatter(pc[:, 3], pc[:, 1])
        axarr[2, 1].scatter(pc_test[:, 3], pc_test[:, 1], color="red")
        axarr[3, 1].scatter(pc[:, 4], pc[:, 1])
        axarr[3, 1].scatter(pc_test[:, 4], pc_test[:, 1], color="red")
        axarr[0, 2].axis('off')
        axarr[1, 2].axis('off')
        axarr[2, 2].scatter(pc[:, 3], pc[:, 2])
        axarr[2, 2].scatter(pc_test[:, 3], pc_test[:, 2], color="red")
        axarr[3, 2].scatter(pc[:, 4], pc[:, 2])
        axarr[3, 2].scatter(pc_test[:, 4], pc_test[:, 2], color="red")
        axarr[0, 3].axis('off')
        axarr[1, 3].axis('off')
        axarr[2, 3].axis('off')
        axarr[3, 3].scatter(pc[:, 4], pc[:, 3])
        axarr[3, 3].scatter(pc_test[:, 4], pc_test[:, 3], color="red")
        plt.savefig(filename)
        Image(filename = filename)

## Lime

In [59]:
class LIME:
    def __init__(self, training, testing, identifier):
        self.training = training
        self.testing = testing
        self.identifier = identifier
        training.clf.fit(training.reduced_data_table, training.predict)
        self.predict_fn = lambda x: training.clf.predict_proba(x).astype(float)
        categorical_features = range(len(training.reduced_feature_patterns))
        categorical_names = {}
        for feature in categorical_features:
            le = sklearn.preprocessing.LabelEncoder()
            le.fit(training.reduced_data_table[:, feature])
            categorical_names[feature] = le.classes_
        explainer = lime.lime_tabular.LimeTabularExplainer(testing.reduced_data_table, verbose=True,
                                                           feature_names=training.reduced_feature_patterns,
                                                           class_names = [str('Low'+ identifier), str('High' + identifier)], 
                                                           categorical_features=categorical_features,
                                                           categorical_names=categorical_names, kernel_width = 3)
        self.explainer = explainer
    def molecule(self, local=False):
        import imp
        indigo = imp.load_source('indigo', 'indigo-python-1.2.3.r0-mac/indigo.py')
        indigo_renderer = imp.load_source('inigo_renderer', 'indigo-python-1.2.3.r0-mac/indigo_renderer.py')
        indigo = indigo.Indigo()
        indigoRenderer = indigo_renderer.IndigoRenderer(indigo)
        def getAtomsActivity (m, patterns):
            matcher = indigo.substructureMatcher(m)
            atom_values = defaultdict(float)
            for pattern, value in patterns:
                try:
                    query = indigo.loadQueryMolecule(pattern)
                    for match in matcher.iterateMatches(query):
                        for qatom in query.iterateAtoms():
                            atom = match.mapAtom(qatom)
                            atom_values[atom.index()] += value / query.countAtoms()
                except:
                    pass
            return atom_values
        def addColorSGroups (m, atom_values):
            min_value = min(atom_values.itervalues())
            max_value = max(atom_values.itervalues())
            centered_value = (min_value + max_value) / 2.
            for atom_index, atom_value in atom_values.iteritems():
                if atom_value < 0.:
                    color = "0, 0, %f" % abs(atom_value / centered_value)
                elif atom_value > 0.:
                    color = "%f, 0, 0" % abs(atom_value / centered_value)
                m.addDataSGroup([atom_index], [], "color", color)
            return min_value, max_value

        def assignColorGroups (m, patterns):
            atom_values = getAtomsActivity(m, patterns)
            min_value, max_value = addColorSGroups(m, atom_values)
            return min_value, max_value
        for count, (id, compound) in enumerate(self.testing.testing_data.compound.iteritems()):
            id_name = id
            print(count, id_name)
            _base = 'pubchem.ncbi.nlm.nih.gov'
            uri = '/rest/pug/compound/cid/' + str(id_name) + '/record/SDF'
            uri = 'http://' + _base + uri
            if not local:
                response = urllib2.urlopen(uri)
                value = response.read().strip().decode().strip('$$$$')
                filename = "data/" + str(id_name) + ".sdf"
                text_file = open(filename, "w")
                text_file.write(value)
                text_file.close()
            row = count
            #Collect explanations from LIME
            exp = self.explainer.explain_instance(self.testing.reduced_data_table[row],
                                                  self.predict_fn,
                                                  num_features=len(self.training.reduced_feature_patterns),
                                                  top_labels=1, verbose=True, num_samples=5000)
            #Load molecule
            if local:
                mol = indigo.iterateSDFile(local)
                m = mol.at(count)
            else:
                mol = indigo.iterateSDFile(filename)
                m = mol.at(0)
            patterns = []
            #Find the local explanation: exp.local_exp[1]
            intercept = exp.intercept.keys()[0]
            local_prob = exp.intercept.values()[0]
            prob = exp.predict_proba[intercept]
            for k, v in exp.local_exp.items():
                for (num, val) in v:
                    print(str(id_name), exp.domain_mapper.exp_feature_names[num], val)
                #Map the explanation to the feature, if it is present in the molecule move forward
                    if float(exp.domain_mapper.feature_values[num]) == 1.:
                        if abs(val) != 0.:
                            patterns.append((self.testing.reduced_feature_patterns[num], val))
                #Draw molecules
            indigo.setOption("render-atom-ids-visible", "false");
            indigo.setOption("render-atom-color-property", "color")
            indigo.setOption('render-coloring', False)
            indigo.setOption('render-comment-font-size', 32)
            indigo.setOption('render-bond-line-width', 2.0)
            indigo.setOption("render-margins", 100, 1);
            indigo.setOption('render-comment', id_name)
            try:
                assignColorGroups(m, patterns)
            except:
                pass
            renderfile = "img/" + str(self.identifier) + str(id_name) + ".png"
            indigoRenderer.renderToFile(m, renderfile)

In [52]:
def run_classifier(training_file, testing_file, roc_file, identifier, train=False, cv=True, visualize=True, lime=True, local=False, delim=""):
    if not train:
        train = Train()
        train.load(training_file, identifier)
        train.reduce_features()
        train.learn()
    if cv:
        CV = CrossValidate(train)
        CV.cross_validation()
        if visualize:
            CV.visualize(roc_file)
    test = Testing()
    test.load(testing_file)
    test.reduce_features(train)
    test.learn(train)
    if visualize:
        viz = VisualizeTesting(train, test)
        viz.pca()
        var_file = delim + "visualized_variance.png"
        viz.viz_explained(var_file)
        xx_file = delim + "visualized_xx.png"
        viz.viz_xx(xx_file)
    if lime:
        lime = LIME(train, test, identifier)
        lime.molecule(local)
    return(train)

In [8]:
training = run_classifier('cetane.txt', "testing_data.txt", 'testing_data.png', 'Cetane', cv=False, train=False, visualize=True, lime=True, delim="testing")

[[ 0.421875    0.578125  ]
 [ 0.59765625  0.40234375]
 [ 0.52148438  0.47851562]
 [ 0.56640625  0.43359375]
 [ 0.52539062  0.47460938]
 [ 0.72460938  0.27539062]
 [ 0.69921875  0.30078125]
 [ 0.51757812  0.48242188]
 [ 0.74023438  0.25976562]
 [ 0.52929688  0.47070312]
 [ 0.44921875  0.55078125]]
['Tetrahydro-3,5-dimethyl-6-propyl-2H-pyran-2-one', '6-Ethyltetrahydro-2H-pyran-2-one', 'SCHEMBL13355779', 'CID_14101663', 'SCHEMBL5585129', '3,6-dimethyloxan-2-one', '2H-Pyran-2-one,tetrahydro-3,5,6-trimethyl-', '648434-43-9', '5,6-dimethyloxan-2-one', 'SCHEMBL13355780', 'SCHEMBL13355731']
(0, '101410820')
(1, '102968')
(2, '14015767')
(3, '14101663')
(4, '15632339')
(5, '19490')
(6, '20717729')
(7, '45095989')
(8, '544628')
(9, '73810187')
(10, '78159702')
[[ 0.66796875  0.33203125]
 [ 0.82617188  0.17382812]
 [ 0.734375    0.265625  ]
 [ 0.68554688  0.31445312]
 [ 0.75195312  0.24804688]
 [ 0.703125    0.296875  ]
 [ 0.54492188  0.45507812]
 [ 0.7421875   0.2578125 ]
 [ 0.74609375  0.253906